# Notes

Linear programming is a set of techniques used in mathematical optimization. We usually use linear programming to maximize/minimize the objective function with some given constraints.

## ```Example 1 (Two Variables)```

### Sample Problem

Maximize $z = x + 2y$

subject to
$\begin{gather}
2x + y \le 20 \\
-4x + 5y \le 10 \\
-x + 2y \ge -2 \\
x \ge 0 \\
y \ge 0
\end{gather}$.

In this example, we are going to find the maximum value of the function $z$ with the given constraints on $x$ and $y$.

In [1]:
from scipy.optimize import linprog

### _Step 1: Define the Coefficients of the Objective Function_

In [2]:
obj_coeff = [-1, -2]

# Note that Python always minimizes the objective function. Hence, we need to transform the problem to "minimizing -x-2y".

### _Step 2: Define the Coefficients of the Constraints_

In [3]:
con_coeff = [
    [2, 1],
    [-4, -5],
    [-1, 2]
]

### _Step 3: Define the Bounds of the Constraints_

In [4]:
con_bd = [20, 10, -2]

### _Step 4: Define the Bounds of the Variables_

In [5]:
x_bd = (0, None)  # lower bound = 0, upper bound = none
y_bd = (0, None)
bd = [x_bd, y_bd]

### _Step 5: Call Out the Solver_

In [6]:
results = linprog(obj_coeff, A_ub = con_coeff, b_ub = con_bd, bounds = bd, method = 'highs')

In [7]:
print('Maximum value: ', -results.fun)  # Note that a negative sign should be added as Python only computes minimum value.
print('x:', results.x[0])
print('y:', results.x[1])

Maximum value:  14.799999999999999
x: 8.4
y: 3.1999999999999993


## ```Example 2 (Three Variables)```

### Sample Problem

Minimize $z = 3x_1 + x_2 - 2x_3$

subject to
$\begin{gather}
5x_1 - x_2 + 3x_3 \le 5 \\
-2x_1 + 4x_3 \le -3 \\
\end{gather}$.

In [8]:
z_cf = [3, 1, -2]
con_cf = [
    [5, -1, 3],
    [-2, 0, 4]
]
con_const = [5, -3]
solu = linprog(z_cf, A_ub = con_cf, b_ub = con_const, method = 'highs')

In [9]:
print('Minimum value: ', solu.fun)
print('x1:', solu.x[0])
print('x2:', solu.x[1])
print('x3:', solu.x[2])

Minimum value:  7.0
x1: 1.5
x2: 2.5
x3: 0.0


## ```Example 3 (Integer Programming)```

Sometime we would like the solution of the optimization to be integers. Hence, we should use integer programming to achieve that. Package "pulp" will be used in the following example, which is more powerful than the solver in package "scipy". "pulp" can solve both ordinary linear programming and integer programming.

### Sample Problem

ABC company provides three products X, Y and Z. Each unit of X requires 30 minutes of processing time on machine A and 50 minutes of processing time on machine B. Each unit of Y requires 20 minutes on machine A and 35 minutes on machine B. Each unit of Z requires 35 minutes on machine A and 10 minutes on machine B.

At the beginning of the week, ABC company has 20 units of X, 70 units of Y and 10 units of Z in stock. Available processing time on machine A for the week is 40 hours and machine B 50 hours. The demand for X, Y and Z are forecasted to be 50 units, 90 units and 20 units. How many product X, Y and Z should ABC company produce in order to maximize the total number of products in stock?

In [10]:
!pip install pulp
import pandas as pd
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 31.5 MB/s eta 0:00:00


In [11]:
from pulp import *

In [12]:
model = LpProblem('Production', LpMaximize)       # LpProblem(<problem_name>, LpMaximize/LpMinimize)
x = LpVariable('Product X', 30, None, LpInteger)  # LpVariable(<variable_name>, <lower_bound>, <upper_bound>, LpContinuous/LpInteger)
y = LpVariable('Product Y', 20, None, LpInteger)
z = LpVariable('Product Z', 10, None, LpInteger)
# Originally, we have three constraints, x+20 >= 50, y+70 >= 90 and z+10 >= 20.
# We may simplify the three constraints to the lower bound of x, y and z.

model += x + 20 + y + 70 + z + 10                 # define the objective functions to be maximized
# We need to maximize total units of products = x + 20 + y + 70 + z + 10.

model += 30*x + 20*y + 35*z <= 2400, 'Machine A Time'  # constraints 1: Machine A Time (constraint name)
model += 50*x + 35*y + 10*z <= 3000, 'Machine B Time'  # constraints 2

In [13]:
model.solve()

1

In [14]:
for v in model.variables():
    print(v.name, '=', v.varValue)

Product_X = 30.0
Product_Y = 36.0
Product_Z = 22.0


# Exercise

1. A factory produces 2 types of watches, Classic and Sporty. For producing each Classic watch, it requires 4 hours of man-power for assembly, 2 hours for finishing and 1 hour for testing; for each Sporty watch, it requires 3 hours for assembly, 3 hours for finishing and 3 hours for testing. There are 265 hours of man powers for assembly, 120 hours for finishing and 70 hours for testing. If the profits of selling each Classic watch and Sporty watch are $\text{\$}$5000 and \$6500 respectively, find the number of each type of watch that the factory should produce such that the total profit can be maximized.

2. The advertising alternatives for a company include television, radio, and newspaper advertisements. The costs and estimates for audience coverage are given in the following table.

In [ ]:
import pandas as pd
data = {'Television':[2000, 100000], 'Newspaper':[600, 40000], 'Radio':[300, 18000]}
df = pd.DataFrame(data, index = ['Cost per advertisement ($)', 'Audience per advertisement'])
print(df)

                            Television  Newspaper  Radio
Cost per advertisement ($)        2000        600    300
Audience per advertisement      100000      40000  18000


The local newspaper limits the number of weekly advertisements from a single company to ten.
Moreover, in order to balance the advertising among the three types of media, no more than half of the
total number of advertisements should occur on the radio, and at least 10% should occur on television.
The weekly advertising budget is $18,200. How many advertisements should be run in each of the three
types of media to maximize the total audience?